### Step 1: Set Parameters

In [1]:
import numpy as np
import pandas as pd

In [2]:
N_GENOMES = 5 # How many different genomes do you want to use?
GENOME_LEN = 100000 # How many base pairs will be in each genome?

N_GENES = 5000 # How many genes to simulate
GENE_LEN = 200 # How many characters in each gene?
P_MUTATION = 0.1 # Probability of a mutation

### Step 2: Simulate Genomes

In [3]:
# For now, keep genomes as a string
# This will make simulating the genes more convenient due to Numpy's indexing
genomes = np.random.choice(['A', 'C', 'G', 'T'], (N_GENOMES,GENOME_LEN))

### Step 3: Simulate Genes
For each gene, save the text of the mutated gene, index of the source genome, the start index within the genome, and the number of mutations applied

In [4]:
source_g = np.random.choice(N_GENOMES, N_GENES)
start_i = np.random.choice(GENOME_LEN - GENE_LEN, N_GENES)

genes = pd.DataFrame({"source_g": source_g, "start_i": start_i})
print(genes.head())

   source_g  start_i
0         1    63747
1         4    46353
2         2    61214
3         4    28668
4         0    16438


In [5]:
CHOICES = {
    'A': ['C', 'G', 'T'],
    'C': ['A', 'G', 'T'],
    'G': ['A', 'C', 'T'],
    'T': ['A', 'C', 'G'],
}

def mutate(source_g, start_i, genomes):
    gene = (genomes[source_g, start_i:start_i + GENE_LEN]).copy()
    n_mutations = np.random.binomial(GENE_LEN, P_MUTATION)
    mutation_indices = np.random.choice(GENE_LEN, n_mutations)
    for i in mutation_indices:
        gene[i] = np.random.choice(CHOICES[gene[i]])

    mutations = GENE_LEN - (genomes[source_g, start_i:start_i + GENE_LEN] == gene).sum()
    
    return "".join(gene), mutations
    # return gene, mutations

In [6]:
genes["raw_mutation_result"] = genes.apply(lambda row: mutate(row[0], row[1], genomes), axis=1)
genes["full_text"] = genes["raw_mutation_result"].apply(lambda x: x[0])
genes["n_mutations"] = genes["raw_mutation_result"].apply(lambda x: x[1])
genes.drop("raw_mutation_result", axis=1, inplace=True)

# Now that we're done simulating genes, join genome as a string
genomes = np.apply_along_axis(lambda row: "".join(row),1,genomes)

In [7]:
for i, genome in enumerate(genomes):
    fname = "../data/genome_{}_{}_{}_{}_{}_{}.txt".format(N_GENOMES, GENOME_LEN, N_GENES, GENE_LEN, P_MUTATION, i)
    with open(fname, "w") as f:
        f.write(genome)
    genes.source_g.replace(i, fname, inplace=True)

In [8]:
fname = "../data/genes_{}_{}_{}_{}_{}.csv".format(N_GENOMES, GENOME_LEN, N_GENES, GENE_LEN, P_MUTATION)
genes.sort_values("source_g", inplace=True)
genes.to_csv(fname, index=False, header=False)

print("Saved to test data to {}".format(fname))

Saved to test data to ../data/genes_5_100000_5000_200_0.1.csv


In [9]:
genes.head()

,source_g,start_i,full_text,n_mutations
2499,../data/genome_5_100000_5000_200_0.1_0.txt,30330,GATACTGGCTAACCAAAACCTTATAAAAGGCACGCCATTAGATGAC...,21
3029,../data/genome_5_100000_5000_200_0.1_0.txt,26534,GTTGCGGGGATGACAACAGGATGAGTACTGTATCTGCCGCGGGTAT...,21
3036,../data/genome_5_100000_5000_200_0.1_0.txt,61349,CGTGGCTTCCTGCAGACAACACTGTCATCGAGGGTTACAGATGAAC...,19
3043,../data/genome_5_100000_5000_200_0.1_0.txt,18499,TATCCATCACAGTAACTCGGTTATACAGGATTCTTCGCTCGGCTCC...,18
3048,../data/genome_5_100000_5000_200_0.1_0.txt,97006,CGGCGTGTTAACACGCTGCACCAGCCCGATCTACATGCTCACCTAT...,15


### Step 4: Evaluate
How often does the longest common substring align with the gene's location?

In [10]:
from difflib import SequenceMatcher

In [11]:
# return predicted start index of gene in genome
matcher = SequenceMatcher(autojunk=False)


def predict_location(gene, source_g):
    matcher.set_seqs(gene, genomes[int(source_g[-5:-4])])
    match = matcher.find_longest_match(0, GENE_LEN, 0, GENOME_LEN)
    return match.b - match.a

predict_location(genes.full_text[0], genes.source_g[0])

63747

In [12]:
from datetime import datetime

start = datetime.now()
predicted_location = genes.head(100).apply(lambda row: predict_location(row["full_text"], row["source_g"]), axis=1)
done = datetime.now()

print("Calculated accuracy rate for 100 genes in {} seconds".format((done - start).total_seconds()))
(predicted_location == genes.head(100).start_i).value_counts()

Calculated accuracy rate for 100 genes in 182.719385 seconds


True    100
dtype: int64